In [ ]:
# Cài đặt các thư viện cần thiết (chạy cell này khi cần)
!pip install xgboost
!pip install pandas
!pip install numpy
!pip install vectorbt
!pip install matplotlib
!pip install scikit-learn

In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
# Đọc dữ liệu OHLCV, chuyển 'timestamp' về datetime và đặt làm index
# 'price' là chuỗi giá đóng cửa dùng cho backtest
df = pd.read_csv("ETHUSDT.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').set_index('timestamp')
price = df['close']


In [79]:
# Tính log return
df['log_ret'] = np.log(df['close']).diff()

# Tính các chỉ báo kỹ thuật
window = 1000
# Average True Range (sử dụng high-low trung bình)
df['atr'] = (df['high'] - df['low']).rolling(window).mean()
df['atr_pct'] = df['atr'] / df['close']
# Phạm vi cao/thấp trong window
df['range_high'] = df['high'].rolling(window=window, min_periods=100).max()
df['range_low']  = df['low'].rolling(window=window, min_periods=100).min()
df['range_width'] = (df['range_high'] - df['range_low']) / df['close']
# Khối lượng trung bình và z-score
df['vol_mean'] = df['volume'].rolling(window).mean()
df['vol_std'] = df['volume'].rolling(window).std()
df['vol_z'] = (df['volume'] - df['vol_mean']) / df['vol_std']

# Tính Moving Averages: ma150 và ma500
# ma150: trung bình động 150 kỳ
# ma500: trung bình động 500 kỳ
df['ma150'] = df['close'].rolling(window=150, min_periods=150).mean()
df['ma500'] = df['close'].rolling(window=500, min_periods=500).mean()

# Xác định vùng tích lũy/pha tán và breakout/breakdown
# is_accumulation: biên hẹp + ATR thấp + volume giảm
# is_distribution: biên hẹp + ATR thấp + volume tăng
# breakout: giá đóng cửa vượt range_high của chu kỳ trước
# breakdown: giá đóng cửa phá vỡ range_low (sử dụng buffer)
df['is_accumulation'] = (
    (df['range_width'] < df['range_width'].quantile(0.5)) &
    (df['atr_pct'] < df['atr_pct'].quantile(0.5)) &
    (df['vol_z'] < 0.5)
)
df['breakdown'] = df['close'] < df['range_low'].shift(1) * 0.98  # 2% breakdown buffer

df['is_distribution'] = (
    (df['range_width'] < df['range_width'].quantile(0.5)) &
    (df['atr_pct'] < df['atr_pct'].quantile(0.5)) &
    (df['vol_z'] > 0.5)
)
df['breakout'] = df['close'] > df['range_high'].shift(1) * 1.02  # 2% breakout buffer

# Nhãn tương lai: dùng tỷ suất log so với median trong rolling window
horizon = 50 
df['future_ret'] = np.log(df['close'].shift(-horizon) / df['close'])
df['label'] = (
    df['future_ret'] > df['future_ret'].rolling(200).median()
).astype(int)

# Các feature dùng cho mô hình
features = [
    'atr_pct',
    'range_width',
    'vol_z',
    'log_ret'
]

# Chia train/test theo năm
X_train = df[df.index.year<2024][features]
y_train = df[df.index.year<2024]['label']
X_test = df[df.index.year>=2024][features]
y_test = df[df.index.year>=2024]['label']

# Khởi tạo mô hình XGBoost cho bài toán phân loại nhị phân
model = XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, random_state=42)

# Huấn luyện mô hình trên tập train
model.fit(X_train, y_train)
# Gán xác suất dự đoán cho các hàng thuộc tập test
df.loc[df.index.year>=2024, 'ml_prob'] = model.predict_proba(X_test)[:, 1]

# Tạo DataFrame test copy để tiện thao tác khi xây tín hiệu
test_mask = df.index.year >= 2024
test_df = df[test_mask].copy()
bull_regime = test_df['ma150'] > test_df['ma500']
bear_regime = test_df['ma150'] < test_df['ma500']

In [80]:
# Xác định tín hiệu vào/ra cho cả long và short (dùng test_df)
# long_entries: vùng tích lũy + breakout + xác suất ML > ngưỡng
# short_entries: vùng phân phối + breakdown + xác suất ML thấp
long_exits = (
    test_df['is_distribution'] &     # regime bắt đầu xấu
    (test_df['close'] < test_df['range_high'])  # breakout thất bại 
    & test_df['breakdown']
)

short_exits = (
    test_df['is_accumulation'] &
    (test_df['close'] > test_df['range_low'])
    & test_df['breakout']
)

breakdown_confirmed = (
    test_df['breakdown'] &
    (test_df['vol_z'] > 0.3)
)
breakout_confirmed = ( test_df['breakout'] & (test_df['vol_z'] > 0.5) )
long_score = (
    0.3 * breakout_confirmed.astype(int) + 
    0.3 * test_df['is_accumulation'].shift(1).fillna(0).astype(int) +
    0.2 * (test_df['ml_prob'] >0.2)
) 

long_entries = ( (long_score >= 0.5) & ~test_df['is_distribution'] )

long_entries = long_entries & bull_regime


short_score = (
    0.4 * breakdown_confirmed.astype(int) +
    0.3 * test_df['is_distribution'].shift(1).fillna(0).astype(int) +
    0.2 * (test_df['ml_prob'] < 0.45).astype(int)
)

short_entries = (
    (short_score >= 0.5) &
    ~test_df['is_accumulation']
)


short_entries = short_entries & bear_regime

# Stop-loss tỷ lệ tính theo ATR
sl_pct = (3 * test_df['atr']) / test_df['close']
sl_pct = sl_pct.clip(0.02, 0.2)

# Tạo portfolio bằng vectorbt
trade_value = 0.05 * 1_000_000  # 5% NAV
pf = vbt.Portfolio.from_signals(
    close=price[test_mask],
    entries=long_entries,
    exits=long_exits,
    size=trade_value,
    size_type='value',  
    short_entries=short_entries,
    short_exits=short_exits,
    sl_stop=sl_pct,
    fees=0.0005,
    init_cash=1_000_000,
    cash_sharing=True,
    accumulate=True,
    freq='30min'
)

In [81]:
# Hiển thị các thống kê hiệu suất của portfolio
stats = pf.stats()

stats

Start                                2024-01-01 00:00:00
End                                  2025-08-15 06:00:00
Period                                 592 days 06:30:00
Start Value                                    1000000.0
End Value                                  2052662.83595
Total Return [%]                              105.266284
Benchmark Return [%]                          102.466384
Max Gross Exposure [%]                      13924.056205
Total Fees Paid                             45919.086773
Max Drawdown [%]                                34.58068
Max Drawdown Duration                  493 days 05:30:00
Total Trades                                         125
Total Closed Trades                                  124
Total Open Trades                                      1
Open Trade PnL                             945990.743005
Win Rate [%]                                   47.580645
Best Trade [%]                                 36.269547
Worst Trade [%]                

In [82]:
# Hiển thị biểu đồ các giao dịch trên biểu đồ giá
pf.plot_trades().show()

In [83]:
print(f"Annually Return: {pf.annual_returns()}")

Annually Return: timestamp
2024-01-01    0.355484
2024-12-31    0.514339
Freq: 365D, Name: group, dtype: float64
